In [1]:
import os
import time

import dash
import dash_bootstrap_components as dbc
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

ModuleNotFoundError: No module named 'prophet'

In [3]:
def run_prophet(
    df,
    weekly_seasonality=False,
    start=2012,
    end=2016,
    uncertainty_samples=200,
    periods=180,
):
    yr = pd.to_datetime(df.ds).dt.year
    yr_df = df[(yr >= start) & (yr <= end)]
    with suppress_stdout_stderr():
        m = Prophet(
            weekly_seasonality=weekly_seasonality,
            uncertainty_samples=uncertainty_samples,
            growth="flat",
        )
        m.fit(yr_df)

    future = m.make_future_dataframe(periods=periods)
    forecast = m.predict(future)

    return m, forecast

In [24]:
vegyData = pd.read_csv(r"C:\Users\Omar\Downloads\VisualData2project\scale-ai-templates-master\scale-ai-templates-master\apps\time-series-forecasting\archive\ProductPriceIndex.csv")


In [19]:
vegyData.head()

,productname,date,farmprice,atlantaretail,chicagoretail,losangelesretail,newyorkretail,averagespread
0,Strawberries,2019-05-19,$1.16,$2.23,$1.70,$1.99,$2.54,82.33%
1,Romaine Lettuce,2019-05-19,$0.35,$1.72,$2.00,$1.69,$1.99,428.57%
2,Red Leaf Lettuce,2019-05-19,$0.32,$1.84,$1.84,$1.69,$1.89,467.19%
3,Potatoes,2019-05-19,$1.50,$5.32,$5.14,$3.99,$6.22,244.50%
4,Oranges,2019-05-19,$0.41,$1.42,$1.45,$1.34,$2.05,281.71%


In [12]:
vegyData['year'] = pd.to_datetime(vegyData['date']).dt.year
vegyData = vegyData[vegyData["farmprice"]!='$']
vegyData["farmprice"].replace('[\$,]', '', regex=True).astype(float)
vegyData.groupby(['year', 'productname'])['farmprice'].mean().reset_index()

C:\Users\Omar\AppData\Local\Temp\ipykernel_10200\3478136954.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vegyData['year'] = pd.to_datetime(vegyData['date']).dt.year


TypeError: agg function failed [how->mean,dtype->object]

In [31]:
vegyData["productname"].unique()

array(['Strawberries', 'Romaine Lettuce', 'Red Leaf Lettuce', 'Potatoes',
       'Oranges', 'Iceberg Lettuce', 'Green Leaf Lettuce', 'Celery',
       'Cauliflower', 'Carrots', 'Cantaloupe', 'Broccoli Crowns',
       'Avocados', 'Broccoli Bunches', 'Asparagus', 'Flame Grapes',
       'Thompson Grapes', 'Honeydews', 'Tomatoes', 'Plums', 'Peaches',
       'Nectarines'], dtype=object)

In [29]:
vegyData['year'] = pd.to_datetime(vegyData['date']).dt.year
# Convert 'price' column to numeric type, coerce non-numeric values to NaN
vegyData = vegyData[vegyData["farmprice"]!='$']
vegyData['farmprice'] = vegyData["farmprice"].replace('[\$,]', '', regex=True).astype(float)

# Drop rows with NaN values in 'price' column

vegyData.dropna(subset=['farmprice'], inplace=True)

# Group by year and productname and calculate the average price
avg_prices = vegyData.groupby(['year', Avocados])['farmprice'].mean().reset_index()
avg_prices

,year,productname,farmprice
0,1999,Avocados,1.010000
1,1999,Broccoli Bunches,0.342000
2,1999,Broccoli Crowns,0.458000
3,1999,Cantaloupe,0.256250
4,1999,Carrots,0.117778
...,...,...,...
440,2019,Oranges,0.417000
441,2019,Potatoes,1.650000
442,2019,Red Leaf Lettuce,0.488000
443,2019,Romaine Lettuce,0.442000


In [15]:
yr = pd.to_datetime(vegyData.date).dt.year

In [17]:
yr

0        2019
1        2019
2        2019
3        2019
4        2019
         ... 
15761    1999
15762    1999
15763    1999
15764    1999
15765    1999
Name: date, Length: 15766, dtype: int32

In [18]:
yr_df = vegyData[(yr >= 1999) & (yr <= 2019)]

In [19]:
yr_df

,productname,date,farmprice,atlantaretail,chicagoretail,losangelesretail,newyorkretail,averagespread
0,Strawberries,2019-05-19,$1.16,$2.23,$1.70,$1.99,$2.54,82.33%
1,Romaine Lettuce,2019-05-19,$0.35,$1.72,$2.00,$1.69,$1.99,428.57%
2,Red Leaf Lettuce,2019-05-19,$0.32,$1.84,$1.84,$1.69,$1.89,467.19%
3,Potatoes,2019-05-19,$1.50,$5.32,$5.14,$3.99,$6.22,244.50%
4,Oranges,2019-05-19,$0.41,$1.42,$1.45,$1.34,$2.05,281.71%
...,...,...,...,...,...,...,...,...
15761,Broccoli Crowns,1999-10-24,$0.29,$1.46,$0.00,$0.97,$1.15,208.62%
15762,Broccoli Bunches,1999-10-24,$0.23,$1.21,$0.00,$0.84,$1.10,242.39%
15763,Cauliflower,1999-10-24,$0.19,$1.01,$0.00,$0.75,$0.92,252.63%
15764,Carrots,1999-10-24,$0.12,$0.64,$0.00,$0.84,$0.59,331.25%


In [21]:
yr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15766 entries, 0 to 15765
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   productname       15766 non-null  object
 1   date              15766 non-null  object
 2   farmprice         15766 non-null  object
 3   atlantaretail     15766 non-null  object
 4   chicagoretail     15766 non-null  object
 5   losangelesretail  15766 non-null  object
 6   newyorkretail     15766 non-null  object
 7   averagespread     15766 non-null  object
dtypes: object(8)
memory usage: 985.5+ KB


In [26]:
filtered_data = yr_df[(yr_df["productname"]=="Strawberries") & (yr_df["newyorkretail"]!='$')]
filtered_data.head()

,productname,date,farmprice,atlantaretail,chicagoretail,losangelesretail,newyorkretail,averagespread
0,Strawberries,2019-05-19,$1.16,$2.23,$1.70,$1.99,$2.54,82.33%
13,Strawberries,2019-05-12,$0.91,$2.67,$1.89,$2.47,$2.66,166.21%
27,Strawberries,2019-05-05,$0.58,$2.42,$1.89,$2.12,$2.91,302.59%
39,Strawberries,2019-04-28,$0.58,$2.79,$1.95,$2.38,$2.96,334.48%
52,Strawberries,2019-04-21,$0.69,$2.92,$2.04,$2.38,$3.16,280.43%


In [22]:
filtered_data = yr_df[yr_df["productname"]=="Strawberries"].dropna()
filtered_data["newyorkretail"] = filtered_data["newyorkretail"].replace('[\$,]', '', regex=True).astype(float)
df = filtered_data[["date", "newyorkretail"]].rename(columns={"date": "ds", "newyorkretail": "y"})

ValueError: could not convert string to float: ''

In [5]:
def run_forecast(category, state, seasonality, periods, uncertainty_samples, years):
    t0 = time.time()
    col = f"('{category}', '{state}')"
    df = walmart[["date", col]].rename(columns={"date": "ds", col: "y"})

    m, forecast = run_prophet(
        df,
        weekly_seasonality=seasonality,
        periods=periods,
        uncertainty_samples=uncertainty_samples,
        start=years[0],
        end=years[1],
    )

    # Plot figures
    margin = dict(l=30, r=30, t=80, b=30)

    fig_forecast = plot_plotly(m, forecast).update_layout(
        title="Forecasting with Prophet",
        width=None,
        height=None,
        margin=margin,
    )

    fig_components = plot_components_plotly(m, forecast).update_layout(
        title="Seasonal Components", width=None, height=None, margin=margin
    )

    t1 = time.time()
    print(f"Training and Inference time: {t1-t0:.2f}s.")

    return fig_forecast, fig_components

NameError: name 'category' is not defined